In [1]:
import cma, mo, pickle

# Defining a callable multiobjective function:

class FitFun:
    def __init__(self, *args):
        self.callables = args
    def __call__(self, x):
        return [f(x) for f in self.callables]
fitness = FitFun(cma.ff.sphere, lambda x: cma.ff.sphere(x-1)) 

# Set parameters
dimension = 10
num_kernels = 5

# Instantiating `Sofomore` with `cma-es`, via the `get_cmas` factory function:

list_of_solvers = mo.get_cmas(num_kernels * [dimension * [0.3]], 0.2)
moes = mo.Sofomore(list_of_solvers, reference_point = [11,11])


In [2]:
class Order:
    """
    `Order(moes)` is a function that takes an index `i` as argument, and returns
    the opposite contributing hypervolume of `moes.kernels[i].objective_values`
    into `moes.front`.
    """
    def __init__(self, moes):
        self.moes = moes
    def __call__(self,i):
        if self.moes.kernels[i].objective_values not in self.moes.front:
            # meaning that the point is not nondominated
            return 0
        else: # the point is nondominated: the (opposite) contributing hypervolume is a non zero value
            index = self.moes.front.bisect_left(self.moes.kernels[i].objective_values)
            return - self.moes.front.contributing_hypervolume(index)
             

In [3]:
#while not moes.stop():
for i in range(100):
    solutions = moes.ask()
    objective_values = [fitness(x) for x in solutions]
    moes.tell(solutions, objective_values)
    moes.disp()
    moes.order = Order(moes)
pickle.dump(moes, open('saved-mocma-object.pkl', 'wb'))
print('saved')
del moes  # let's start fresh

Iterat #Fevals   Hypervolume   axis ratios   sigmas   min&max stds
                                  (median)  (median)    (median)
    1     10 6.161000000000001e+01 1.0e+00 2.00e-01  2e-01  2e-01
    2     20 6.592335425875837e+01 1.0e+00 2.00e-01  2e-01  2e-01
    3     30 6.722966527270513e+01 1.0e+00 1.91e-01  2e-01  2e-01
  100   1000 9.598830431979563e+01 1.6e+00 8.87e-02  7e-02  8e-02
saved


In [4]:
moes = pickle.load(open('saved-mocma-object.pkl', 'rb'))

In [5]:
[moes.order(i) for i in range(5)]

[Fraction(-8545998748809195668318358556665, 2535301200456458802993406410752),
 Fraction(-2470265143276750688931611749539, 633825300114114700748351602688),
 Fraction(-13436973697942462279613818375475, 5070602400912917605986812821504),
 Fraction(-15759010397027829393534255011435, 5070602400912917605986812821504),
 Fraction(-10624668229784718367734051033075, 2535301200456458802993406410752)]

In [6]:
moes.optimize(fitness, iterations = 400)

  200   2000 9.721055364943868e+01 1.9e+00 2.59e-02  2e-02  2e-02
  300   3000 9.723786908683871e+01 1.9e+00 7.72e-03  5e-03  5e-03
  400   4000 9.724066565151536e+01 2.0e+00 1.79e-03  9e-04  1e-03
  500   5000 9.724106411795985e+01 2.2e+00 8.62e-04  3e-04  4e-04


In [7]:
type(moes.order(0))

fractions.Fraction